In [14]:

from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT


In [15]:
#借助包gym_super_mario_bros创建
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [16]:
def process_frame(frame):
    if frame is not None:
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) #图像转换
        frame = cv2.resize(frame, (84, 84))[None, :, :] / 255. #裁剪合适大小，并归一化
        return frame
    else:
        return np.zeros((1, 84, 84))

In [17]:
#使用gym的wrapper函数对游戏视频进行录像（由于notebook不支持display，我们录像后播放观看）
# from gym import wrappers#
# env = wrappers.Monitor(env,"./gym-results", force=True)

这玩意在windows下不好使，网上找到也都不怎么好用。索性就自己写了一个

In [18]:
import cv2  
def game_show(state):
# state为传入的图像信息，通常保存在env.step的第一个返回值中
    cv2.imshow("game",state)
    cv2.waitKey(1)


下面这一步在最新版本的gym中可能会出现报错，问题在于env.step的返回值个数在不同的版本中不同。介于还在学习过程，建议先将gym的版本拉低

In [19]:
done = True
for step in range(5000):
    # print(step)
    if done:
        state = env.reset()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    # state = process_frame(state)
    game_show(state)
cv2.destroyAllWindows()
env.close()

6
4
3
2
1
1
3
0
1
1
4
2
3
0
2
5
2
1
3
4
3
2
0
4
3
0
1
1
6
1
4
0
2
0
5
6
6
5
4
3
4
6
1
3
3
1
6
0
2
0
3
6
3
0
4
5
0
5
3
4
3
1
2
2
5
1
3
0
4
1
2
6
6
4
3
3
1
6
2
1
0
1
6
4
2
0
2
1
1
4
0
3
4
5
5
1
0
3
6
2
4
3
1
5
2
4
5
4
0
5
1
4
1
5
2
5
5
3
1
2
5
5
1
0
0
0
4
3
4
1
0
4
2
4
3
2
3
0
5
6
1
1
4
5
6
3
0
4
2
0
0
2
5
3
3
0
3
4
2
0
1
5
1
5
3
5
0
2
2
3
2
4
6
1
6
2
5
1
5
2
4
2
6
3
6
4
3
0
2
2
0
0
3
5
0
0
0
0
4
2
1
3
5
2
0
6
6
0
2
2
5
4
1
0
6
5
4
4
6
6
5
4
1
3
6
6
3
3
2
6
0
1
1
3
6
3
3
6
4
4
1
4
2
2
0
1
6
2
4
3
0
3
1
5
5
2
2
4
3
4
0
5
0
4
2
1
5
2
5
5
1
1
6
1
4
4
5
2
4
4
6
1
3
1
2
3
0
3
6
2
2
3
3
5
2
6
0
5
6
0
0
0
2
5
6
1
5
5
4
1
4
6
4
3
3
4
4
2
3
5
4
3
5
6
2
1
6
2
2
3
1
5
1
5
4
5
0
4
3
1
2
2
3
3
3
5
6
2
5
2
0
4
1
0
1
3
5
2
2
4
6
4
1
1
1
6
0
3
2
1
3
2
1
2
0
5
6
6
3
6
4
3
2
4
5
4
1
2
2
0
1
6
4
1
4
4
1
3
1
2
6
1
6
0
2
1
1
6
4
0
4
3
5
4
1
5
5
2
5
4
2
3
2
1
1
5
0
0
1
1
5
5
0
4
1
5
1
1
1
1
2
5
0
2
3
6
2
0
3
2
5
5
0
5
6
0
3
3
2
0
4
3
1
5
2
6
3
2
6
1
6
4
6
1
0
2
2
2
4
4
5
4
5
1
1
0
1
3
1
5
6
6
1
2
5
1
2
1
6
5


强化学习执行step动作的返回一般是标准的，所以这里的返回同前面的倒立摆，动作执行后，环境返回四个变量（obj:新的观测值（一帧rgb图片）、reward：执行该动作获得的奖励值（在游戏中的得分）、done:回合是否结束（你控制的小人是不是死了，对应回合结束）、info:额外信息（比如'life': 2，剩余2条命等））

In [20]:
import gym_super_mario_bros
from gym.spaces import Box
from gym import Wrapper
from nes_py.wrappers import JoypadSpace#BinarySpaceToDiscreteSpaceEnv
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY
import cv2
import numpy as np
import subprocess as sp

In [21]:

class CustomSkipFrame(Wrapper):
    def __init__(self, env, skip=4):
        super(CustomSkipFrame, self).__init__(env)
        self.observation_space = Box(low=0, high=255, shape=(4, 84, 84))
        self.skip = skip

    def step(self, action):
        total_reward = 0
        states = []
        state, reward, done, info = self.env.step(action)
        for i in range(self.skip):
            if not done:
                state, reward, done, info = self.env.step(action)
                total_reward += reward
                states.append(state)
            else:
                states.append(state)
        states = np.concatenate(states, 0)[None, :, :, :]
        return states.astype(np.float32), reward, done, info

    def reset(self):
        state = self.env.reset()
        states = np.concatenate([state for _ in range(self.skip)], 0)[None, :, :, :]
        return states.astype(np.float32)



In [22]:
class CustomReward(Wrapper):
    def __init__(self, env=None):
        super(CustomReward, self).__init__(env)
        self.observation_space = Box(low=0, high=255, shape=(1, 84, 84))
        self.curr_score = 0

    def step(self, action):
        state, reward, done, info = self.env.step(action)
        state = process_frame(state)
        reward += (info["score"] - self.curr_score) / 40.
        self.curr_score = info["score"]
        if done:
            if info["flag_get"]:
                reward += 50
            else:
                reward -= 50
        return state, reward / 10., done, info

    def reset(self):
        self.curr_score = 0
        return process_frame(self.env.reset())

In [23]:
#至此，我们完成了超级玛丽环境的自定义，封装如下：
def create_train_env(world, stage, action_type, output_path=None):
    env = gym_super_mario_bros.make("SuperMarioBros-{}-{}-v0".format(world, stage))
    if action_type == "right":
        actions = RIGHT_ONLY
    elif action_type == "simple":
        actions = SIMPLE_MOVEMENT
    else:
        actions = COMPLEX_MOVEMENT
    env = JoypadSpace(env, actions)
    env = CustomReward(env)
    env = CustomSkipFrame(env)
    return env, env.observation_space.shape[0], len(actions)

In [24]:
#测试一下
custom_env = create_train_env(1,1,'simple')
print(custom_env)


(<CustomSkipFrame<CustomReward<JoypadSpace<TimeLimit<SuperMarioBrosEnv<SuperMarioBros-1-1-v0>>>>>>, 4, 7)
